# Классификация лиц в масках/без с помощью передачи обучения из предварительно обученной сети EfficientNetB3

In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
import pathlib
import os

#Используем буферизованную предварительную выборку для загрузки изображений с диска без блокировки ввода-вывода
AUTOTUNE = tf.data.experimental.AUTOTUNE

data_root_orig='/kaggle/input/face-mask-dataset/data'

class_names = ['with_mask', 'without_mask']
BATCH_SIZE = 32
IMG_SIZE = (160, 160)

output_path='/kaggle/working/'

In [2]:
def preprocess_image(path, image_size):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [image_size[0], image_size[1]])
    return image


In [3]:
def show_img(train_dataset):
    import matplotlib.pyplot as plt
    data_augmentation = tf.keras.Sequential([
        tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
        tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
    ])
    for image, _ in train_dataset.take(1):
        plt.figure(figsize=(10, 10))
        first_image = image[0]
        for i in range(9):
            ax = plt.subplot(3, 3, i + 1)
            augmented_image = data_augmentation(tf.expand_dims(first_image, 0))
            plt.imshow(augmented_image[0] / 255)
            plt.axis('off')

In [4]:
def custom(class_names, data_root_orig, batch, image_size):
    data_root = pathlib.Path(data_root_orig)
    all_image_paths = list(data_root.rglob('*/*.jpg'))
    all_image_paths = [str(path) for path in all_image_paths if pathlib.Path(path).is_file()]

    image_count = len(all_image_paths)

    label_names = []
    for path in all_image_paths:
        if pathlib.Path(path).parent.name == class_names[0]:
            label_names.append(0)
        else:
            label_names.append(1)

    path_ds = tf.data.Dataset.from_tensor_slices(all_image_paths)
    image_ds = path_ds.map(lambda path: preprocess_image(path, image_size), num_parallel_calls=AUTOTUNE)
    label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(label_names, tf.int64))
    image_label_ds = tf.data.Dataset.zip((image_ds, label_ds))

    # Установка размера буфера перемешивания, равного набору данных, гарантирует
    # полное перемешивание данных.
    ds = image_label_ds.shuffle(buffer_size=image_count)
    ds = ds.repeat()
    ds = ds.batch(batch)

    size_ds = image_count
    train_size = int(0.8*size_ds)
    test_size = int(0.2*size_ds)
    train_ds = ds.take(train_size)
    test_ds = ds.skip(train_size)
    test_ds = ds.take(test_size)

    return train_ds, test_ds

In [5]:
train_dataset, validation_dataset = custom(
    class_names = class_names,
    data_root_orig = data_root_orig,
    batch = BATCH_SIZE,
    image_size = IMG_SIZE
)

посмотрим на первые 9 изображений и меток из обучающего набора

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_dataset.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

Необходимо определить, сколько пакетов данных доступно в наборе проверки, используя tf.data.experimental.cardinality , а затем переместим 20% из них в набор тестов.

In [ ]:
val_batches = tf.data.experimental.cardinality(validation_dataset)
test_dataset = validation_dataset.take(val_batches // 5)
validation_dataset = validation_dataset.skip(val_batches // 5)
print('Number of validation batches: %d' % tf.data.experimental.cardinality(validation_dataset))
print('Number of test batches: %d' % tf.data.experimental.cardinality(test_dataset))

Используем буферизованную предварительную выборку для загрузки изображений с диска без блокировки ввода-вывода

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

Искусственно введем разнообразие образцов, применяя случайные, но реалистичные преобразования к обучающим изображениям, такие как поворот и горизонтальный поворот. Это помогает раскрыть модель различным аспектам обучающих данных и уменьшить переобучение .

In [ ]:
data_augmentation=tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip(mode='horizontal'),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
    tf.keras.layers.experimental.preprocessing.RandomZoom(.5, .2),
    tf.keras.layers.experimental.preprocessing.RandomTranslation(height_factor=0.1, width_factor=0.1),
    tf.keras.layers.experimental.preprocessing.RandomContrast(factor=0.1),
])

проверим внесенное разнообразие слоев - визуализируем первое изображение, учитывая разные повороты

In [ ]:
for image, _ in train_dataset.take(1):
  plt.figure(figsize=(10, 10))
  first_image = image[0]
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    augmented_image = data_augmentation(tf.expand_dims(first_image, 0))
    plt.imshow(augmented_image[0] / 255)
    plt.axis('off')

## Создадим базовую модель из предварительно обученных сверток

Для извлечения функций модели возьмем  слой «узкое место» -  элементы слоя  сохраняют большую универсальность по сравнению с последним / верхним слоем.

In [ ]:
image_shape=IMG_SIZE+(3,)
base_model=tf.keras.applications.EfficientNetB3(input_shape=image_shape,
                                    include_top=False,
                                    weights='imagenet',
                                    drop_connect_rate=0.4)

In [ ]:
image_batch, label_batch = next(iter(train_dataset))
feature_batch = base_model(image_batch)
print(feature_batch.shape)

Извлечение признаков

На этом шаге заморозим сверточную базу, созданную на предыдущем шаге, и будем использовать ее в качестве средства извлечения признаков. Плюс добавим поверх него классификатор и обучим классификатор верхнего уровня.

Заморозить сверточную базу

Перед компиляцией и обучением модели важно заморозить сверточную базу. Замораживание (путем установки layer.trainable = False) предотвращает обновление весов в данном слое во время обучения. EfficientNet имеет много уровней, поэтому установка для trainable флага всей модели значения False заморозит их все.

In [ ]:
base_model.trainable = False

In [ ]:
base_model.summary()

Добавить заголовок классификации

Чтобы сгенерировать прогнозы на основе блока функций, используем слой tf.keras.layers.GlobalAveragePooling2D

In [ ]:
model=tf.keras.Sequential()
model.add(base_model)
model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(tf.keras.layers.Dense(1))

In [ ]:
model.summary()

Создадим модель, объединив вместе слои

In [ ]:
input=tf.keras.Input(image_shape)
x=data_augmentation(input)
x=base_model(x,training=False)
x=tf.keras.layers.GlobalAveragePooling2D()(x)
x=tf.keras.layers.Dropout(0.2)(x)
x=tf.keras.layers.BatchNormalization()(x)
output=tf.keras.layers.Dense(1)(x)
model=tf.keras.Model(input,output)

In [ ]:
model.summary()

Скомпилируем модель

Скомпилируем модель перед ее обучением

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

Обучим модель

In [ ]:
init_epochs=10

history=model.fit(train_dataset,
          epochs=init_epochs,
          validation_data=validation_dataset
         )

Кривые обучения

Давайте посмотрим на кривые обучения точности / потери обучения и проверки при использовании базовой предобученной модели  в качестве экстрактора фиксированных функций.

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

В меньшей степени это также связано с тем, что метрики обучения сообщают среднее значение за эпоху, тогда как метрики валидации оцениваются после эпохи, поэтому метрики валидации видят модель, которая тренировалась немного дольше.

Тонкая настройка

В эксперименте по извлечению признаков мы тренировали только несколько слоев поверх базовой модели EfficientNet V2. Веса предварительно обученной сети не обновлялись во время обучения.

Один из способов еще больше повысить производительность - это обучить (или «точно настроить») веса верхних слоев предварительно обученной модели вместе с обучением добавленного нами классификатора. Процесс обучения заставит настраивать веса с общих карт объектов на объекты, связанные специально с набором данных.

Разморозить верхние слои модели

Все, что вам нужно сделать, это разморозить base_model и сделать нижние слои base_model . Затем нам следует перекомпилировать модель (необходимо, чтобы эти изменения вступили в силу) и возобновить обучение.

In [ ]:
base_model.trainable = True


In [ ]:
print("Number of layers in the base model: ", len(base_model.layers))

fine_tune_at = 100

for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False

Скомпилируйте модель

Поскольку мы тренируем гораздо большую модель и хотим повторно адаптировать предварительно обученные веса, на этом этапе важно использовать более низкую скорость обучения. В противном случае наша модель может очень быстро переобучиться

In [ ]:
base_learning_rate=0.0001
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer = tf.keras.optimizers.RMSprop(lr=base_learning_rate/10),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
len(model.trainable_variables)

Продолжить обучение модели

In [ ]:
fine_tune_epochs = 10
total_epochs =  init_epochs + fine_tune_epochs

history_fine = model.fit(train_dataset,
                         epochs=total_epochs,
                         initial_epoch=history.epoch[-1],
                         validation_data=validation_dataset)

Давайте посмотрим на кривые обучения точности / потери обучения и проверки при тонкой настройке последних нескольких уровней базовой предобученной модели  и обучении классификатора поверх нее. Потеря проверки намного выше, чем потеря тренировки, поэтому мы можем получить некоторое переобучение.

Мы также можем столкнуться с переобучением, поскольку новый обучающий набор относительно невелик и похож на исходные наборы данных предобученной модели.

После точной настройки модель достигает около 99% точности на проверочном наборе.

In [ ]:
acc += history_fine.history['accuracy']
val_acc += history_fine.history['val_accuracy']

loss += history_fine.history['loss']
val_loss += history_fine.history['val_loss']

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.ylim([0.8, 1])
plt.plot([init_epochs-1,init_epochs-1],
          plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.ylim([0, 1.0])
plt.plot([init_epochs-1,init_epochs-1],
         plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

Оценка и прогноз

Наконец, мы можем проверить работоспособность модели на новых данных с помощью набора тестов.

In [ ]:
loss, accuracy = model.evaluate(test_dataset)
print('Test accuracy :', accuracy)

И теперь мы готовы использовать эту модель, чтобы предсказать конечную цель - лицо в маске или без

In [ ]:
image_batch, label_batch = test_dataset.as_numpy_iterator().next()
predictions = model.predict_on_batch(image_batch).flatten()

# Apply a sigmoid since our model returns logits
predictions = tf.nn.sigmoid(predictions)
predictions = tf.where(predictions < 0.5, 0, 1)

print('Predictions:\n', predictions.numpy())
print('Labels:\n', label_batch)

plt.figure(figsize=(10, 10))
for i in range(9):
  ax = plt.subplot(3, 3, i + 1)
  plt.imshow(image_batch[i].astype("uint8"))
  plt.title(class_names[predictions[i]])
  plt.axis("off")